In [1]:
!pip install numpy==1.19.3
!pip install fastparquet
!pip install python-snappy
# !pip install pymorphy2
!pip install scikit-learn==0.23.2
!pip install gensim==3.8.3

     |████████████████████████████████| 14.9MB 296kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


     |████████████████████████████████| 122kB 11.7MB/s 
     |████████████████████████████████| 61kB 3.9MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.5.0-cp37-cp37m-linux_x86_64.whl size=213345 sha256=897ba58d1a984ef1c8524c8922b74162d758ff7419cebfc2608b939a96f907c9
  Stored in directory: /root/.cache/pip/wheels/97/0e/da/e2122965947ab2bfbc20e31a968e998fc8932f7fcb0ed78ad0
  Created wheel for thrift: filename=thrift-0.13.0-cp37-cp37m-linux_x86_64.whl size=348139 sha256=fd6f38ef01d1de3830a1396f683c83f72c7ec031f2e17ffcad59071cecb3bdde
  Stored in directory: /root/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built fastparquet thrift
     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 6.8MB 10.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |█████████████████████████████

In [ ]:
cur_submit = 'Submit10'
data_train_path = '../data_fusion_train.parquet'
w2v_path = f'w2v.model'
ft_path = 'ft.model'

# Importing

In [ ]:
# Data Analysis
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

# Service tools
from tqdm.notebook import tqdm, trange
import logging, os, pickle, json
tqdm.pandas()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Work with text
import re
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download('stopwords')

# Sklearn Pipelines & Preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import *

# Models & Metrics
from sklearn.cluster import KMeans
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

# метрика задачи
def score(y_true, y_val):
    return f1_score(y_true, y_val, average='weighted')

# датафрейм, содержащий категорию n
def samples(n):
    return df[df.category_id == n]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rodion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Читает и создаёт два файла: 
* backup.csv - неразмеченный датасет, 
* backup_label.csv - размеченный датасет

In [ ]:
def read_save(read_path):
    df = pd.read_parquet(read_path)[['item_name', 'category_id']]
    df = df.drop_duplicates('item_name')
    df[df.category_id == -1]['item_name'].to_csv('backup.csv', index=False)
    df[df.category_id != -1].to_csv('backup_label.csv', index=False)
    logging.warning('To save RAM restart Notebook and then skip this cell')

df = read_save(data_train_path)

# Preprocessing

In [ ]:
from preprocessing import normalize
df = pd.read_csv('backup.csv')
df.dropna(inplace=True)
df.item_name = normalize(df.item_name)

E:\DataFusionHack\Submit10\preprocessing.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  return ser \


# FastText

Подготовка данных

In [ ]:
df.item_name = df.item_name.apply(lambda x: [i for i in x.split() if len(i) > 1])

In [ ]:
df = [l for l in df.item_name.values if l]

Обучение

In [ ]:
model = FastText(df, size=200, window=35, min_count=2, workers=12, iter=30, sg=1, bucket=500_000)

2021-03-13 23:58:41,778 : INFO : resetting layer weights
2021-03-13 23:58:44,732 : INFO : collecting all words and their counts
2021-03-13 23:58:44,733 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-13 23:58:44,747 : INFO : PROGRESS: at sentence #10000, processed 48972 words, keeping 13589 word types
2021-03-13 23:58:44,760 : INFO : PROGRESS: at sentence #20000, processed 98177 words, keeping 20901 word types
2021-03-13 23:58:44,774 : INFO : PROGRESS: at sentence #30000, processed 147526 words, keeping 26404 word types
2021-03-13 23:58:44,788 : INFO : PROGRESS: at sentence #40000, processed 197923 words, keeping 31384 word types
2021-03-13 23:58:44,803 : INFO : PROGRESS: at sentence #50000, processed 248272 words, keeping 35787 word types
2021-03-13 23:58:44,817 : INFO : PROGRESS: at sentence #60000, processed 298648 words, keeping 39756 word types
2021-03-13 23:58:44,832 : INFO : PROGRESS: at sentence #70000, processed 348531 words, keeping 43651 wo

Сохранение модели

In [ ]:
model.save('ft.model')

2021-03-14 00:38:05,405 : INFO : saving FastText object under ft.model, separately None
2021-03-14 00:38:05,406 : INFO : storing np array 'vectors' to ft.model.wv.vectors.npy
2021-03-14 00:38:06,541 : INFO : storing np array 'vectors_vocab' to ft.model.wv.vectors_vocab.npy
2021-03-14 00:38:07,807 : INFO : storing np array 'vectors_ngrams' to ft.model.wv.vectors_ngrams.npy
2021-03-14 00:38:10,528 : INFO : not storing attribute vectors_norm
2021-03-14 00:38:10,529 : INFO : not storing attribute vectors_vocab_norm
2021-03-14 00:38:10,529 : INFO : not storing attribute vectors_ngrams_norm
2021-03-14 00:38:10,530 : INFO : not storing attribute buckets_word
2021-03-14 00:38:10,531 : INFO : storing np array 'syn1neg' to ft.model.trainables.syn1neg.npy
2021-03-14 00:38:11,801 : INFO : storing np array 'vectors_vocab_lockf' to ft.model.trainables.vectors_vocab_lockf.npy
2021-03-14 00:38:12,715 : INFO : storing np array 'vectors_ngrams_lockf' to ft.model.trainables.vectors_ngrams_lockf.npy
2021-

## Model optimization

In [ ]:
from pathlib import Path
list(map(str,Path('.').glob('*.npy')))

['ft.model.trainables.syn1neg.npy',
 'ft.model.trainables.vectors_ngrams_lockf.npy',
 'ft.model.trainables.vectors_vocab_lockf.npy',
 'ft.model.wv.vectors.npy',
 'ft.model.wv.vectors_ngrams.npy',
 'ft.model.wv.vectors_vocab.npy']

In [ ]:
for file_ones in ['ft.model.trainables.vectors_ngrams_lockf.npy',
                   'ft.model.trainables.vectors_vocab_lockf.npy']:
    print(file_ones)
    spam = np.load(file_ones)
    print('min/max values before:', spam.min(), spam.max())
    spam = spam.astype('float16')
    print('min/max values after:', spam.min(), spam.max())
    with open(file_ones, 'wb') as f:
        np.save(f, spam)
    print('__________')

ft.model.trainables.vectors_ngrams_lockf.npy
min/max values before: 1 1
min/max values after: 1.0 1.0
__________
ft.model.trainables.vectors_vocab_lockf.npy
min/max values before: 1 1
min/max values after: 1.0 1.0
__________


In [ ]:
for file_vectors in ['ft.model.trainables.syn1neg.npy',
                     'ft.model.wv.vectors.npy',
                     'ft.model.wv.vectors_ngrams.npy',
                     'ft.model.wv.vectors_vocab.npy']:
    print(file_vectors)
    spam = np.load(file_vectors)
    print('min/max values before:', spam.min(), spam.max())
    spam = spam.astype('float16')
    print('min/max values after:', spam.min(), spam.max())
    with open(file_vectors, 'wb') as f:
        np.save(f, spam)
    print('__________')

ft.model.trainables.syn1neg.npy
min/max values before: -2.9071774 2.544437
min/max values after: -2.906 2.545
__________
ft.model.wv.vectors.npy
min/max values before: -2.9149797 2.7795186
min/max values after: -2.914 2.78
__________
ft.model.wv.vectors_ngrams.npy
min/max values before: -10.478997 11.849249
min/max values after: -10.48 11.85
__________
ft.model.wv.vectors_vocab.npy
min/max values before: -18.041256 17.898159
min/max values after: -18.05 17.89
__________


### Vector Averaging

In [ ]:
def word_averaging(wv, words):
    mean = np.zeros((wv.vector_size,))
    
    for word in words:
        mean += wv.get_vector(word)

    mean = gensim.matutils.unitvec(mean)
    return mean

def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list])

# Labeled data

## Importing & normalize

In [ ]:
from preprocessing import normalize
df = pd.read_csv('backup_label.csv')
df.dropna(inplace=True)
df.item_name = normalize(df.item_name)

Скачивание модели

In [ ]:
model = FastText.load('ft.model')

2021-03-14 10:13:53,069 : INFO : loading FastText object from ft.model
2021-03-14 10:13:53,533 : INFO : loading wv recursively from ft.model.wv.* with mmap=None
2021-03-14 10:13:53,534 : INFO : loading vectors from ft.model.wv.vectors.npy with mmap=None
2021-03-14 10:13:53,571 : INFO : loading vectors_vocab from ft.model.wv.vectors_vocab.npy with mmap=None
2021-03-14 10:13:53,610 : INFO : loading vectors_ngrams from ft.model.wv.vectors_ngrams.npy with mmap=None
2021-03-14 10:13:53,715 : INFO : setting ignored attribute vectors_norm to None
2021-03-14 10:13:53,716 : INFO : setting ignored attribute vectors_vocab_norm to None
2021-03-14 10:13:53,716 : INFO : setting ignored attribute vectors_ngrams_norm to None
2021-03-14 10:13:53,717 : INFO : setting ignored attribute buckets_word to None
2021-03-14 10:13:53,717 : INFO : loading vocabulary recursively from ft.model.vocabulary.* with mmap=None
2021-03-14 10:13:53,717 : INFO : loading trainables recursively from ft.model.trainables.* with

In [ ]:
df.item_name = df.item_name.apply(lambda x: [i for i in x.split() 
                                             if len(i) > 1])
X_wv = word_averaging_list(model.wv, df.item_name)
X_wv = X_wv.astype('float16')
y = df.category_id

# Fitting

In [ ]:
pipe = make_pipeline(
    StandardScaler(),
    SVC(random_state=0),
)

In [ ]:
%%time
cross_val_score(pipe, 
                X_wv, 
                y, 
                cv=StratifiedKFold(5, 
                                   shuffle=True, 
                                   random_state=0), 
                scoring='f1_weighted',
                n_jobs=5
)

Wall time: 5min 40s


array([0.86110397, 0.85244531, 0.85905061, 0.85158041, 0.86331367])

# Сохранение модели классификатора

In [ ]:
pipe.fit(X_wv, y)
pickle.dump(pipe, open(f'clf_task1', 'wb'))